Grupo 7

In [ ]:
! gdown --id 1Ui7dofcyVVJlqPSqnJjmufPKnZxw5ZAj #properatti.csv
! gdown --id 1I2V15Sf2mUqXQUQvExfvzipDxnWMqvXM #provincia.shx
! gdown --id 1Im6hNqTDb1zaCX4Lo9czGaMmqgzvDj9S #ar.csv
! gdown --id 1RCIHoEfk07OiSZw5PtOxVdj4aiHxQJpC #provincia.shp
! gdown --id 1EcVqccbr0qdJtuyzOHBRtSkkJZlxf6_Z #provincia.prj
! gdown --id 1r3ElDulmkTvdGwL6zuIWLZ08C8bM4kqw #provincia.dbf
! gdown --id 11VXNu7X_BIfB-2ZtStlxQmHoL_OBdafk #provincia.cpg

In [ ]:

!pip install --upgrade geopandas pyshp shapely descartes
!pip install --upgrade unidecode
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from unidecode import unidecode


In [ ]:
# traigo el dataset Properatti

df_properatti= pd.read_csv('/content/properatti.csv',sep=',')
print(df_properatti.shape)
df_properatti.head(5)

In [ ]:
df_properatti['state_name'].value_counts()

In [ ]:
# nombre columnas
print(df_properatti.dtypes)


In [ ]:
# primer valor de cada columna
for column in df_properatti.columns:
    print(f"{column}: {df_properatti[column].iloc[0]}")

In [ ]:
# importo el Mapa de Argentina CON info por provincias

data_location = "/content/provincia.shx"
mapa_argentina = gpd.read_file(data_location)

# importo el Mapa (2) de Argentina SIN info por provincias

mundo = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
mapa_argentina_2 = mundo[mundo.name == "Argentina"]

mapa_argentina.head(24)

In [ ]:
# este mapa trae info de provincias.
mapa_argentina.plot()

In [ ]:
# este mapa no trae info de provincias (mas practico para ploteo general de todas las propiedades)

mapa_argentina_2.plot()

In [ ]:
# ejemplo con una provincia

mapa_san_luis = mapa_argentina[mapa_argentina.fna == "Provincia de San Luis"]
mapa_san_luis.plot()

In [ ]:
# Dataframe con la info sobre geonames_id

location_geonames = '/content/AR.csv'
data_geonames = gpd.read_file(location_geonames, sep='\t')

# convierte a float64 la columna 'GEONAMEID' que es un string

data_geonames['GEONAMEID'] = pd.to_numeric(data_geonames['GEONAMEID'], errors='coerce').astype(float)

print(data_geonames.shape)
data_geonames.head(10)

In [ ]:
print(data_geonames.dtypes)

In [ ]:
# filtro solo el nombre de la provincia y creo una columna llamada 'provincia'

pattern = re.compile(r'/([^/]+)$')

def cambiar_nombre(timezone):
    match = re.search(pattern, timezone)
    return match.group(1).replace('_', ' ') if match else None

data_geonames['provincia'] = data_geonames['TIMEZONE'].apply(lambda x: cambiar_nombre(x))
data_geonames.head(5)

data_geonames.provincia.isnull().sum()

data_geonames[data_geonames.provincia.isnull()].head(5)

In [ ]:
# cuento cuantas filas hay en df_properatti que no tienen ninguna info geografica

mask_lat_lon = np.logical_and(df_properatti['lat'].isnull(), df_properatti['lon'].isnull())
mask_latlon = df_properatti['lat-lon'].isnull()
mask_geonames = df_properatti['geonames_id'].isnull()
combined_mask = mask_lat_lon & mask_geonames


geografia_filtered_df = df_properatti.loc[mask_lat_lon]
print("nulls en 'lat' y 'lon': " f"{geografia_filtered_df.shape}")
geografia_filtered_df = df_properatti.loc[mask_latlon]
print("nulls en 'lat-lon': " f"{geografia_filtered_df.shape}")
# 'lat', 'lon' y 'lat-lon' tienen la misma info

geografia_filtered_df = df_properatti.loc[mask_geonames]
print("nulls en 'geonames_id': " f"{geografia_filtered_df.shape}")
geografia_filtered_df = df_properatti.loc[combined_mask]
print("ninguna informacion: " f"{geografia_filtered_df.shape}")




In [ ]:
# Chequeo si falta algun dato de lugar(pais provincia ciudad)

df_properatti.place_with_parent_names.isnull().sum()

# Todos tienen algun tipo de informacion de lugar(pais provincia ciudad)

In [ ]:
# Filtro los acentos
columns_2 = ['place_with_parent_names']
columns_1 = ['NAME', 'ASCIINAME']
# Create a function to remove accents from a string
def remove_accents(text):
    return unidecode(text) if pd.notna(text) else text

# Apply the function to specified columns

df_properatti[columns_2] = df_properatti[columns_2].applymap(remove_accents)
data_geonames[columns_1] = data_geonames[columns_1].applymap(remove_accents)

In [ ]:
# Busca un string
substring_to_search = 'Tucuman'

# Creo mascara booleana
substring_mask = data_geonames['NAME'].str.contains(substring_to_search, case=False)
print(data_geonames[substring_mask].shape)
data_geonames[substring_mask]

In [ ]:
# Renombro las columnas 'state_name', 'place_name' y 'country_name'

"""df_properatti.rename(columns={'state_name': 'provincia'}, inplace=True)
df_properatti.rename(columns={'place_name': 'ciudad'}, inplace=True)
df_properatti.rename(columns={'country_name': 'pais'}, inplace=True)
df_properatti.head(10)"""

In [ ]:
# Limpio la columna 'place_with_parent_name' y creo columnas con info de 'provincia', 'ciudad' y 'barrio'

import re

pattern = re.compile(r'\|(?P<pais>[^|]+)\|(?P<provincia>[^|]+)(?:\|(?P<ciudad>[^|]+))?(?:\|(?P<barrio>[^|]+))?')

def extract_parts(location_info):
    match = pattern.match(location_info)
    return {'provincia': match.group('provincia') if match else None, 'ciudad': match.group('ciudad') if match else None, 'barrio': match.group('barrio') if match else None}

df_properatti[['provincia', 'ciudad', 'barrio']] = df_properatti['place_with_parent_names'].apply(
    lambda x: pd.Series(extract_parts(x))
)




In [ ]:
# Capital Federal queda dentro de la Provincia de Buenos Aires

def clean_capital(row):
    if row['provincia'] == 'Capital Federal':
        row['provincia'] = 'Buenos Aires'
        row['barrio'] = row['ciudad']
        row['ciudad'] = 'Ciudad Autónoma de Buenos Aires'
    return row

df_properatti = df_properatti.apply(clean_capital, axis=1)
df_properatti.head(5)

In [ ]:
df_properatti['provincia'].value_counts()

In [ ]:
# Uso regex para limpiar Bs.As. y demas formas de Buenos Aires.
pattern = re.compile(r'(?P<provincia>Bs\.As\.|Buenos Aires)\s(?P<partido>.+)')

def match_location(location_info):
    if pd.isna(location_info):  # Check for missing values
        return pd.Series({'provincia': None, 'partido': None})

    match = pattern.match(location_info)
    if match:
        provincia = 'Buenos Aires'
        partido = match.group('partido')
        return pd.Series({'provincia': provincia, 'partido': partido})
    else:
        return pd.Series({'provincia': location_info, 'partido': None})


df_properatti[['provincia', 'partido']] = df_properatti['provincia'].apply(match_location)

In [ ]:
df_properatti[['provincia','partido']].value_counts()

In [ ]:
df_properatti[df_properatti['provincia'] == 'Buenos Aires']['ciudad'].value_counts()

In [ ]:
df_properatti['ciudad'].value_counts()

In [ ]:
df_properatti[df_properatti[['lat', 'lon']].isnull().any(axis=1)][['lat', 'lon']].shape


In [ ]:
# busca los valores de 'geonames_id' en el dataframe 'data_geonames'
# cuando los encuentra, se trae el valor de las columnas 'LATITUDE' y 'LONGITUDE'

def normalize_geonameid(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        geoname_id = row['geonames_id']
        if geoname_id in data_geonames['GEONAMEID'].values:
            matching_row = data_geonames[data_geonames['GEONAMEID'] == geoname_id]
            if not matching_row.empty:
                row.loc['lat'] = matching_row['LATITUDE'].iloc[0]
                row.loc['lon'] = matching_row['LONGITUDE'].iloc[0]

    return row

df_properatti = df_properatti.apply(normalize_geonameid, axis=1)

In [ ]:
df_properatti[df_properatti[['lat', 'lon']].isnull().any(axis=1)][['lat', 'lon']].shape

In [ ]:
# Si  no hay info de 'lat' o 'lon', me traigo los valores 'LATITUDE' y 'LONGITUDE' de data_geonames que matcheen con la ciudad

def find_ciudad_province_in_data_geonames(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        ciudad = row['ciudad']
        matching_rows = data_geonames[data_geonames['NAME'].isin([ciudad])]

        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]
            row['lat'] = pd.to_numeric(matching_row['LATITUDE']).astype(float)
            row['lon'] = pd.to_numeric(matching_row['LONGITUDE']).astype(float)

    return row

df_properatti = df_properatti.apply(find_ciudad_province_in_data_geonames, axis=1)

In [ ]:
print(df_properatti[pd.isnull(df_properatti['lat']) & pd.isnull(df_properatti['lon'])][['lat', 'lon']].shape)

In [ ]:
# Si  no hay info de 'lat' o 'lon', me traigo los valores 'LATITUDE' y 'LONGITUDE' de data_geonames que matcheen con la provincia

def find_ciudad_province_in_data_geonames(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        provincia = row['provincia']
        matching_rows = data_geonames[data_geonames['provincia'].isin([provincia])]

        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]
            row['lat'] = pd.to_numeric(matching_row['LATITUDE']).astype(float)
            row['lon'] = pd.to_numeric(matching_row['LONGITUDE']).astype(float)

    return row

df_properatti = df_properatti.apply(find_ciudad_province_in_data_geonames, axis=1)

In [ ]:
print(df_properatti[pd.isnull(df_properatti['lat']) & pd.isnull(df_properatti['lon'])][['lat', 'lon']].shape)
df_properatti[pd.isnull(df_properatti['lat']) & pd.isnull(df_properatti['lon'])]['title'].value_counts()

In [ ]:
def fix_tucuman(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        row['ciudad'] = 'San Miguel de Tucuman'
        row['lat'] = -26.82414
        row['lon'] = -65.22260

    return row

df_properatti = df_properatti.apply(fix_tucuman, axis=1)

In [ ]:
# Filtro los que no tienen info geografica (no hay match en data_geonames)

mask_not_null = np.logical_and(~pd.isnull(df_properatti['lat']),~pd.isnull(df_properatti['lon']))

df_properatti_for_plot = df_properatti[mask_not_null]

df_properatti_for_plot.shape

In [ ]:
# creo un geoDataFrame con la info geografica
geometry = gpd.points_from_xy(pd.to_numeric(df_properatti_for_plot.lon), pd.to_numeric(df_properatti_for_plot.lat))
geometry

df_properatti = gpd.GeoDataFrame(df_properatti_for_plot, geometry=geometry)


# outlier corregido

df_properatti.at[9761, 'geometry'] = Point(-64.00000, -32.00000)


In [ ]:
#GeoDataFrame final con toda la info de geografia, sin valores nulos

df_properatti_final_geografia = df_properatti[['title','place_with_parent_names','provincia','ciudad','barrio','partido','geometry']]
print("Shape: " , df_properatti_final_geografia.shape)
print("total nulos: ", df_properatti_final_geografia['geometry'].isnull().sum())
df_properatti_final_geografia.sample(10)



In [ ]:
# Mapa de Argentina
ax = mapa_argentina_2.plot(color='white', edgecolor='black')

# Sobre las ciudades superpone el mapa del país
df_properatti.plot(ax=ax, color='red', markersize=0.05)

plt.show()

In [ ]:
# filtro los puntos que estan dentro de 'mapa_san_luis'

contained_points = df_properatti[df_properatti.geometry.within(mapa_san_luis.unary_union)]

# plot del mapa_san_luis
ax = mapa_san_luis.plot(figsize=(10, 10), color='lightgray', edgecolor='black')
contained_points.plot(ax=ax, color='red', marker='o', markersize=5, alpha=0.5)
plt.title('Propiedades en San Luis')
plt.show()